In [1276]:
import pandas as pd

In [1278]:
df=pd.read_csv('File_path')

In [1280]:
f= open(r"C:\Users\sahil\OneDrive\Documents\Gen AI\Gen AI Key\New_Key_301.txt")
OPENAI_API_KEY = f.read()



In [1282]:
from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini")

In [1283]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
[
    ("system", '''You are an expert data scientist with strong Python and pandas skills. 
    You are given access to a DataFrame called `{df}`. 
    The user may ask any question related to this data. 
    and you need to just give the python codes and no other thing is required. i don't want to see ```python and ... in starting and end. 
    you need to import all essential lib when you genrate the code
    one more thing say you are creating some code like
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.to_period('M')
    average_revenue_per_month = df.groupby('Month')['Revenue'].mean()

    after this you need to give average_revenue_per_month, as we need to call the fuction.
'''),
    ("human", "{user_input}"),
]
)


In [1284]:
chat_template.input_variables

['df', 'user_input']

In [1285]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [1286]:
chain = chat_template | chat_model | output_parser

In [1287]:
raw_input= {'df':df, 'user_input': 'avag revenue based on categories duing each month'}


In [1288]:
def execute_llm_code(raw_input, dataframe):
    # Get the code from the LLM
    code = chain.invoke({"df": dataframe, "user_input": raw_input})
    
    # Create a local namespace to execute the code
    local_namespace = {"df": dataframe, "pd": pd}
    
    # Execute the code
    try:
        print("Executing code:")
        print("--------------")
        print(code)
        print("--------------")
        
        # Execute the code in the local namespace
        exec(code, globals(), local_namespace)
        
        # Get the last variable name in the code
        lines = code.strip().split("\n")
        last_line = lines[-1].strip()
        
        # Check if the last line is just a variable name
        if last_line in local_namespace and not last_line.startswith(("import", "from", "#", "df[", "print")):
            print("\nResult:")
            print(local_namespace[last_line])
            return local_namespace[last_line]
        else:
            # Try to evaluate the last line
            try:
                result = eval(last_line, globals(), local_namespace)
                print("\nResult:")
                print(result)
                return result
            except:
                print("\nNo explicit result to display.")
                return None
    except Exception as e:
        print(f"Error executing code: {str(e)}")
        return None

In [1289]:
def interactive_data_analysis(dataframe):
    print("Data Analysis Assistant")
    print("Type 'exit' to quit")
    
    while True:
        # Get user query using input function
        query = input("\nWhat would you like to know about the data? ")
        
        # Check for exit command
        if query.lower() in ['exit', 'quit', 'q']:
            print("Exiting data analysis assistant.")
            break
        
        # Process the query
        result = execute_llm_code(query, dataframe)

In [ ]:
interactive_data_analysis(df)

Data Analysis Assistant
Type 'exit' to quit
